In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, BatchNormalization
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
# train test split 
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv("Data/preprocessed_data.csv")
df.head(3)

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,...,spark,aws,excel,sql,django,hadoop,java,db,tableau,azure
0,0,Data Scientist,$74K - $139K,Intro (Use Font Arial 12):\nAs a Data Scientis...,4.4,Applied Information Sciences,"Chevy Chase, MD",501 to 1000 Employees,1982,Unknown,...,0,0,0,1,0,0,0,1,0,1
1,1,Web Developer,$41K - $83K,We are the fastest growing employer of emergin...,4.0,Revature,"Tampa, FL",1001 to 5000 Employees,2003,Unknown,...,0,0,0,0,0,0,0,0,0,0
2,2,Data Analyst,$43K - $88K,"HR Data Analyst - Direct Hire - Buffalo, NY\nO...",4.1,Systems Personnel,"Niagara Falls, NY",1 to 50 Employees,1996,Unknown,...,0,0,0,1,0,0,0,0,1,0


In [9]:
stemmer = PorterStemmer()
tokenizer = RegexpTokenizer("[a-z']+")

# preprocess text
def get_tokens(description):
    text_tokens = tokenizer.tokenize(description)
    stemmed_tokens = [stemmer.stem(x) for x in text_tokens]
    return stemmed_tokens

vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,1), tokenizer=get_tokens)

transformed_text = vectorizer.fit_transform(df['Job Description'])


C:\Users\Eisha\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  warni

In [10]:
max_words = 10000
max_len = 1500
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(df['Job Description'])
sequences = tok.texts_to_sequences(df['Job Description'])
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(transformed_text, df['avg_salary'], test_size=0.3, random_state=52)